Names, base price and profile websites of players sold and unsold in IPL 2022 auction from the website: (Retained players are not included)
https://www.espncricinfo.com/story/2023-ipl-auction-list-of-sold-unsold-players-1350272

Names, auction price and profile websites of players sold in IPL 2022 auction (Retained players are included) from the website:
'https://www.espncricinfo.com/auction/content/site/ipl2022

In [4]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

C:\Users\Kalyani Gohokar\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Getting the column names for the dataset

In [53]:
website = 'https://www.espncricinfo.com/cricketers/rohit-sharma-34102'

result = requests.get(website)
content = result.text
soup = BeautifulSoup(content, 'lxml')

#print(soup.prettify())

In [54]:
head = soup.find_all(class_='ds-bg-fill-content-alternate ds-text-left ds-text-right')

In [55]:
head_ls = []

for i in head:
    head_ls.append(i.get_text(strip=True, separator=' '))

In [56]:
head_ls

['Format Mat Inns NO Runs HS Ave BF SR 100s 50s 4s 6s Ct St',
 'Format Mat Inns Balls Runs Wkts BBI BBM Ave Econ SR 4w 5w 10w']

In [57]:
#Get columns
columns = ['Player'] + head_ls[0].split() + head_ls[1].split()

In [58]:
columns.remove('Format')

In [59]:
columns.remove('Format')

In [60]:
columns.insert(1, 'Role')

In [61]:
columns

['Player',
 'Role',
 'Mat',
 'Inns',
 'NO',
 'Runs',
 'HS',
 'Ave',
 'BF',
 'SR',
 '100s',
 '50s',
 '4s',
 '6s',
 'Ct',
 'St',
 'Mat',
 'Inns',
 'Balls',
 'Runs',
 'Wkts',
 'BBI',
 'BBM',
 'Ave',
 'Econ',
 'SR',
 '4w',
 '5w',
 '10w']

In [14]:
df = pd.DataFrame(columns = columns)

The player statistics for each player are on different websites. So we first find the links for the players.

In [15]:
#Website 1 (using the package beautiful scoop)
website = 'https://www.espncricinfo.com/story/ipl-2022-auction-the-list-of-sold-and-unsold-players-1300689'

result = requests.get(website)
content = result.text
soup = BeautifulSoup(content, 'lxml')

In [16]:
box = soup.find('article', class_ ="ds-mb-4 ds-text-typo ci-story")
#box.find_all('a', href=True)

In [17]:
links = [link['href'] for link in box.find_all('a', href=True)]
print(links)

['/genre/news-1', '/author/himanshu-agrawal-758', 'https://www.espncricinfo.com/auction/content/site/ipl2022', 'https://www.espncricinfo.com/live-blog/live-blog-ipl-2022-auction-1300707', 'https://www.espncricinfo.com/player/aman-khan-1119011', 'https://www.espncricinfo.com/player/david-willey-308251', 'https://www.espncricinfo.com/player/fabian-allen-670013', 'https://www.espncricinfo.com/player/luvnith-sisodia-1155253', 'https://www.espncricinfo.com/player/aryan-juyal-1130300', 'https://www.espncricinfo.com/player/sai-sudharsan-1151288', 'https://www.espncricinfo.com/player/siddarth-kaul-326017', 'https://www.espncricinfo.com/player/daryl-mitchell-381743', 'https://www.espncricinfo.com/player/rassie-van-der-dussen-337790', 'https://www.espncricinfo.com/player/vicky-ostwal-1292520', 'https://www.espncricinfo.com/player/nathan-coulter-nile-261354', 'https://www.espncricinfo.com/player/james-neesham-355269', 'https://www.espncricinfo.com/player/umesh-yadav-376116', 'https://www.espncric

In [18]:
linkss = links[4:-12]

Next, we loop over all the links to scrap the required T20 statistic for each player and add it to a dataframe

In [21]:
for link in linkss:
    result = requests.get(link)
    content = result.text
    soup = BeautifulSoup(content, 'lxml')

    head = soup.find_all(class_='ds-bg-fill-content-alternate ds-text-left ds-text-right')
    head_ls = [] #getting the column names
    for i in head:
        head_ls.append(i.get_text(strip=True, separator=' '))

    Format = soup.find_all(class_="ds-min-w-max ds-text-right ds-capitalize ds-font-bold !ds-text-left ds-whitespace-nowrap")
    Mat = soup.find_all(class_="ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right ds-capitalize ds-text-right")
    Runs_wk = soup.find_all(class_="ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right ds-capitalize ds-text-right ds-font-bold")

    T_role = soup.find_all(class_ ="ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3")
    T_role_ls = []
    for i in T_role:
        T_role_ls.append(i.get_text(strip=True, separator=' '))
        
    if ('Playing Role' in T_role_ls):
        id = T_role_ls.index('Playing Role')

        role_ = soup.find_all('h5')
        role_ls = []
        for i in role_:
            role_ls.append(i.get_text(strip=True, separator=' '))
        role = role_ls[id]
    else:
        role = '-'
            
    name = link[link.rfind('/') + 1:link.rfind('-')]
    
    Format_ls = []

    for i in Format: #getting the format names a player has played
        Format_ls.append(i.get_text(strip=True, separator=' '))

    if 'T20' in Format_ls: #making sure the player has played T20
        n = Format_ls.index('T20')
        Runs_wk_ls = []
        for i in Runs_wk:
            Runs_wk_ls.append(i.get_text(strip=True, separator=' '))
        #print(link)

        Mat_ls = []
        for i in Mat:
            Mat_ls.append(i.get_text(strip=True, separator=' '))

        if (head_ls[0].split()[3] == 'NO'): #Tells if batting is first or bowling #This means batting first
            k = 13*n
            Bowl_t20 = Mat_ls[-12:]
            Bat_t20 = Mat_ls[k:k+13]

            runs = Runs_wk_ls[n]
            wk = Runs_wk_ls[-1] 
        else: 
            k = 12*n #multiply by 12 since in bowling has one column less
            Bat_t20 = Mat_ls[-13:] 
            Bowl_t20 = Mat_ls[k:k+12]

            wk = Runs_wk_ls[n]
            runs = Runs_wk_ls[-1] 

        Ro = [name] + Bat_t20 + Bowl_t20 
        Ro.insert(1, role)
        Ro.insert(5, runs)
        Ro.insert(-8, wk)

        df.loc[linkss.index(link)] = list(Ro)

    else:
        df.loc[linkss.index(link)] = [name, role] + [0]*27

In [22]:
df.head()

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,Runs,Wkts,BBI,BBM,Ave,Econ,SR,4w,5w,10w
0,aman-khan,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,david-willey,Bowling Allrounder,244,180,39,3372,118,23.91,2449,137.68,...,5571,242,4/7,4/7,23.02,7.94,17.3,4,0,0
2,fabian-allen,Bowling Allrounder,92,72,21,935,64*,18.33,661,141.45,...,1726,64,3/24,3/24,26.96,7.68,21.0,0,0,0
3,luvnith-sisodia,Wicketkeeper Batter,15,9,0,124,38,13.77,97,127.83,...,-,-,-,-,-,-,-,-,-,-
4,aryan-juyal,Wicketkeeper Batter,14,12,4,285,71*,35.62,235,121.27,...,-,-,-,-,-,-,-,-,-,-


In [23]:
df['Role'].unique()

array(['Bowler', 'Bowling Allrounder', 'Wicketkeeper Batter',
       'Top order Batter', 'Allrounder', 'Batting Allrounder', 'Batter',
       'Middle order Batter', 'Opening Batter', '-'], dtype=object)

Getting player names and Base Price

In [24]:
#Website 1 (using the package beautiful scoop)
website = 'https://www.espncricinfo.com/story/ipl-2022-auction-the-list-of-sold-and-unsold-players-1300689'

result = requests.get(website)
content = result.text
soup = BeautifulSoup(content, 'lxml')

In [25]:
para = soup.find_all(class_="ds-text-comfortable-m ds-my-4 ci-html-content")

In [26]:
text = []

for i in para:
    text.append(i.text)

In [27]:
name_sal = text[text.index('Sold players')+1:]

In [28]:
name_sal.remove('Unsold players')

In [29]:
names = []
Base_price = []

for i in name_sal:
    names.append(i[0:i.rfind('(') -1])
    Base_price.append(i[i.rfind('(')+1: i.rfind(')')])

In [30]:
BasePrice = []

for i in Base_price:
    BasePrice.append(i[i.find('R')+2:])

In [31]:
df['Names'] = names
df['BasePrice'] = BasePrice

In [37]:
df.head()

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Names,BasePrice
0,aman-khan,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Aman Khan,20 lakh
1,david-willey,Bowling Allrounder,244,180,39,3372,118,23.91,2449,137.68,...,4/7,4/7,23.02,7.94,17.3,4,0,0,David Willey,2 crore
2,fabian-allen,Bowling Allrounder,92,72,21,935,64*,18.33,661,141.45,...,3/24,3/24,26.96,7.68,21.0,0,0,0,Fabian Allen,75 lakh
3,luvnith-sisodia,Wicketkeeper Batter,15,9,0,124,38,13.77,97,127.83,...,-,-,-,-,-,-,-,-,Luvnith Sisodia,20 lakh
4,aryan-juyal,Wicketkeeper Batter,14,12,4,285,71*,35.62,235,121.27,...,-,-,-,-,-,-,-,-,Aryan Juyal,20 lakh


In [40]:
df.columns

Index(['Player', 'Role', 'Mat', 'Inns', 'NO', 'Runs', 'HS', 'Ave', 'BF', 'SR',
       '100s', '50s', '4s', '6s', 'Ct', 'St', 'Mat', 'Inns', 'Balls', 'Runs',
       'Wkts', 'BBI', 'BBM', 'Ave', 'Econ', 'SR', '4w', '5w', '10w', 'Names',
       'BasePrice'],
      dtype='object')

In [43]:
df[df['Wkts'] == 0]

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Names,BasePrice
0,aman-khan,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Aman Khan,20 lakh
18,ramesh-kumar,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ramesh Kumar,20 lakh
51,ashok-sharma,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ashok Sharma,20 lakh
53,arshad-khan,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Mohammad Arshad Khan,20 lakh
54,saurabh-dubey,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Saurabh Dubey,20 lakh
66,aneeshwar-gautam,Bowling Allrounder,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Aneeshwar Gautam,20 lakh
208,rohan-rana,Allrounder,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Rohan Rana,20 lakh
229,sushant-mishra,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Sushant Mishra,20 lakh
263,vasu-vats,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Vasu Vats,20 lakh


In [33]:
df[df['Names'] == 'Matthew Wade']

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Names,BasePrice
46,matthew-wade,Wicketkeeper Batter,231,203,33,4559,130*,26.81,3360,135.68,...,-,-,-,-,-,-,-,-,Matthew Wade,2 crore


In [34]:
df.to_csv('BasePrice.csv')

Thus, we have a dataset with players sold and unsold in the 2022 ipl auction with their T20 statistics and Base price.

Since retained players are not included in the previous dataset, we scrape the player websites from another website.

In [62]:
from selenium import webdriver

url = 'https://www.espncricinfo.com/auction/content/site/ipl2022'

driver = webdriver.Chrome()
driver.get(url)
stages = driver.find_element_by_class_name('stage-scheduled')
driver.close()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_class_name'

In [63]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [64]:
links = [link['href'] for link in soup.find_all('a', href=True)]
print(links)

['/', '/live-cricket-score', 'https://www.espncricinfo.com/live-cricket-score', 'https://www.espncricinfo.com/live-cricket-match-schedule-fixtures', 'https://www.espncricinfo.com/live-cricket-match-results', 'https://www.espncricinfo.com/ci/engine/series/index.html?view=month', 'https://www.espncricinfo.com/ci/engine/series/index.html?view=season', 'https://www.espncricinfo.com/ci/engine/match/index.html?view=calendar', 'https://www.espncricinfo.com/ci/engine/match/scores/desktop.html', 'https://www.espncricinfo.com/ci/content/match/fixtures_futures.html', '/series/women-s-premier-league-2023-1348825', '/series/australia-in-india-2022-23-1348637', '/series/west-indies-in-south-africa-2022-23-1339568', '/series/england-in-bangladesh-2022-23-1351394', '/series/sri-lanka-tour-of-new-zealand-2022-23-1322351', '/series/ireland-in-bangladesh-2022-23-1355710', '/series/afghanistan-v-pakistan-2022-23-1362810', '/series/netherlands-in-zimbabwe-2022-23-1362375', '/series/netherlands-in-zimbabwe-

In [65]:
link = links[links.index('/ci/content/player/1083851.html'): links.index('/ci/content/page/866033.html')]

In [66]:
len(link)

237

In [67]:
dff = pd.DataFrame(columns = columns)

In [68]:
dff

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,Runs,Wkts,BBI,BBM,Ave,Econ,SR,4w,5w,10w


In [69]:
root = 'https://www.espncricinfo.com'
for l in link:
    result = requests.get(f'{root}{l}')
    content = result.text
    soup = BeautifulSoup(content, 'lxml')
    
    head = soup.find_all(class_='ds-bg-fill-content-alternate ds-text-left ds-text-right')
    head_ls = []
    for i in head:
        head_ls.append(i.get_text(strip=True, separator=' '))

    Format = soup.find_all(class_="ds-min-w-max ds-text-right ds-capitalize ds-font-bold !ds-text-left ds-whitespace-nowrap")
    Mat = soup.find_all(class_="ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right ds-capitalize ds-text-right")
    Runs_wk = soup.find_all(class_="ds-w-0 ds-whitespace-nowrap ds-min-w-max ds-text-right ds-capitalize ds-text-right ds-font-bold")

    T_role = soup.find_all(class_ ="ds-text-tight-m ds-font-regular ds-uppercase ds-text-typo-mid3")
    T_role_ls = []
    for i in T_role:
        T_role_ls.append(i.get_text(strip=True, separator=' '))
        
    if ('Playing Role' in T_role_ls):
        id = T_role_ls.index('Playing Role')

        role_ = soup.find_all('h5')
        role_ls = []
        for i in role_:
            role_ls.append(i.get_text(strip=True, separator=' '))
        role = role_ls[id]
    else:
        role = '-'
            
    name = l[l.rfind('/') + 1:l.rfind('-')]
    
    Format_ls = []

    for i in Format: #getting the format names a player has played
        Format_ls.append(i.get_text(strip=True, separator=' '))

    if 'T20' in Format_ls: #making sure the player has played T20
        n = Format_ls.index('T20')
        Runs_wk_ls = []
        for i in Runs_wk:
            Runs_wk_ls.append(i.get_text(strip=True, separator=' '))
        #print(l)

        Mat_ls = []
        for i in Mat:
            Mat_ls.append(i.get_text(strip=True, separator=' '))

        if (head_ls[0].split()[3] == 'NO'): #Tells if batting is first or bowling #This means batting first
            k = 13*n
            Bowl_t20 = Mat_ls[-12:]
            Bat_t20 = Mat_ls[k:k+13]

            runs = Runs_wk_ls[n]
            wk = Runs_wk_ls[-1] 
        else: 
            k = 12*n #multiply by 12 since in bowling has one column less
            Bat_t20 = Mat_ls[-13:] 
            Bowl_t20 = Mat_ls[k:k+12]

            wk = Runs_wk_ls[n]
            runs = Runs_wk_ls[-1] 


        Ro = [name] + Bat_t20 + Bowl_t20 
        Ro.insert(1, role)
        Ro.insert(5, runs)
        Ro.insert(-8, wk)

        dff.loc[links.index(l)] = list(Ro)

    else:
        dff.loc[links.index(l)] = [name, role] + [0]*27

In [70]:
dff.head()

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,Runs,Wkts,BBI,BBM,Ave,Econ,SR,4w,5w,10w
119,1083851.htm,Batting Allrounder,34,31,7,663,64,27.62,501,132.33,...,169,2,1/9,1/9,84.50,7.14,71.0,0,0,0
120,1123073.htm,Wicketkeeper Batter,41,35,5,664,88*,22.13,579,114.68,...,-,-,-,-,-,-,-,-,-,-
121,1155253.htm,Wicketkeeper Batter,15,9,0,124,38,13.77,97,127.83,...,-,-,-,-,-,-,-,-,-,-
122,1159711.htm,Allrounder,62,40,11,719,60*,24.79,561,128.16,...,1196,46,3/7,3/7,26.00,7.16,21.7,0,0,0
123,1176959.htm,Bowling Allrounder,32,9,4,43,13,8.60,37,116.21,...,817,38,4/35,4/35,21.50,7.18,17.9,1,0,0


In [71]:
dff['Role'].unique()

array(['Batting Allrounder', 'Wicketkeeper Batter', 'Allrounder',
       'Bowling Allrounder', 'Top order Batter', 'Bowler',
       'Middle order Batter', 'Opening Batter', 'Batter'], dtype=object)

Getting salaries and names of players

In [72]:
#Getting the price for each player.

from selenium import webdriver

url = 'https://www.espncricinfo.com/auction/content/site/ipl2022'

driver = webdriver.Chrome()
driver.get(url)
stages = driver.find_element_by_class_name('stage-scheduled')
driver.close()

AttributeError: 'WebDriver' object has no attribute 'find_element_by_class_name'

In [73]:
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [74]:
money = soup.find_all(class_="dolr")

In [75]:
salaries = list(money)

In [76]:
rem = salaries[0]
salaries.remove(rem)

In [77]:
def remove_items(test_list, item):
 
    # using list comprehension to perform the task
    res = [i for i in test_list if i != item]
 
    return res

In [78]:
res = remove_items(salaries, rem)
res

[<span class="dolr">30</span>,
 <span class="dolr">340</span>,
 <span class="dolr">20</span>,
 <span class="dolr">240</span>,
 <span class="dolr">20</span>,
 <span class="dolr">20</span>,
 <span class="dolr">1500</span>,
 <span class="dolr">75</span>,
 <span class="dolr">775</span>,
 <span class="dolr">550</span>,
 <span class="dolr">50</span>,
 <span class="dolr">200</span>,
 <span class="dolr">1100</span>,
 <span class="dolr">75</span>,
 <span class="dolr">1075</span>,
 <span class="dolr">700</span>,
 <span class="dolr">25</span>,
 <span class="dolr">1075</span>,
 <span class="dolr">95</span>,
 <span class="dolr">100</span>,
 <span class="dolr">700</span>,
 <span class="dolr">80</span>,
 <span class="dolr">20</span>,
 <span class="dolr">725</span>,
 <span class="dolr">40</span>,
 <span class="dolr">20</span>,
 <span class="dolr">800</span>,
 <span class="dolr">20</span>,
 <span class="dolr">20</span>,
 <span class="dolr">20</span>,
 <span class="dolr">55</span>,
 <span class="dolr">6

In [79]:
#getting the numbers
salaries = []
for i in res:
    salaries.append(str(i)[str(i).rfind('"')+2:str(i).rfind('<')])

In [80]:
dff['Salaries (lakh)']=salaries

In [81]:
dff.head()

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,Wkts,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Salaries (lakh)
119,1083851.htm,Batting Allrounder,34,31,7,663,64,27.62,501,132.33,...,2,1/9,1/9,84.50,7.14,71.0,0,0,0,30
120,1123073.htm,Wicketkeeper Batter,41,35,5,664,88*,22.13,579,114.68,...,-,-,-,-,-,-,-,-,-,340
121,1155253.htm,Wicketkeeper Batter,15,9,0,124,38,13.77,97,127.83,...,-,-,-,-,-,-,-,-,-,20
122,1159711.htm,Allrounder,62,40,11,719,60*,24.79,561,128.16,...,46,3/7,3/7,26.00,7.16,21.7,0,0,0,240
123,1176959.htm,Bowling Allrounder,32,9,4,43,13,8.60,37,116.21,...,38,4/35,4/35,21.50,7.18,17.9,1,0,0,20


In [82]:
#Getting names of players
names_ = soup.find_all('a')

text = []

for i in names_:
    text.append(i.text)

In [83]:
text

['\n\nESPN Cricinfo\n',
 'Live Scores',
 'Live Scores Home',
 'Schedule',
 'Results',
 'Month view',
 'Season view',
 'International calendar',
 'Desktop Scoreboard',
 'Series',
 'WPL 2023',
 'India v Australia',
 'South Africa v West Indies',
 'Bangladesh v England',
 'New Zealand v Sri Lanka',
 'Bangladesh v Ireland',
 'Pakistan v Afghanistan',
 'Zimbabwe v Netherlands',
 'Zimbabwe v Netherlands',
 'IPL 2023',
 'PSL 2023',
 'Legends League',
 'Test Championship',
 "Women's Championship",
 'CWC Super League',
 'CWC League 2',
 '-Future series-',
 '-Archives-',
 'Teams',
 'Australia',
 'Afghanistan',
 'Bangladesh',
 'Ireland',
 'England',
 'Namibia',
 'India',
 'Nepal',
 'New Zealand',
 'Netherlands',
 'Pakistan',
 'Oman',
 'South Africa',
 'PNG',
 'Sri Lanka',
 'Scotland',
 'West Indies',
 'UAE',
 'Zimbabwe',
 'USA',
 'News',
 'News Home',
 'Future of ODIs',
 'Ball-tampering',
 'Technology in cricket',
 'Racism',
 'Features',
 'Features Home',
 'Writers',
 'Photo galleries',
 'The Cri

In [84]:
F = 'Suyash Prabhudessai'
L = 'Dominic Drakes'

names = text[text.index(F):text.index(L)+1]

In [85]:
dff['Names'] = names

In [86]:
dff.head()

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Salaries (lakh),Names
119,1083851.htm,Batting Allrounder,34,31,7,663,64,27.62,501,132.33,...,1/9,1/9,84.50,7.14,71.0,0,0,0,30,Suyash Prabhudessai
120,1123073.htm,Wicketkeeper Batter,41,35,5,664,88*,22.13,579,114.68,...,-,-,-,-,-,-,-,-,340,Anuj Rawat
121,1155253.htm,Wicketkeeper Batter,15,9,0,124,38,13.77,97,127.83,...,-,-,-,-,-,-,-,-,20,Luvnith Sisodia
122,1159711.htm,Allrounder,62,40,11,719,60*,24.79,561,128.16,...,3/7,3/7,26.00,7.16,21.7,0,0,0,240,Shahbaz Ahmed
123,1176959.htm,Bowling Allrounder,32,9,4,43,13,8.60,37,116.21,...,4/35,4/35,21.50,7.18,17.9,1,0,0,20,Akash Deep


In [88]:
df[df['Wkts'] == 0]

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Names,BasePrice
0,aman-khan,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Aman Khan,20 lakh
18,ramesh-kumar,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ramesh Kumar,20 lakh
51,ashok-sharma,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Ashok Sharma,20 lakh
53,arshad-khan,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Mohammad Arshad Khan,20 lakh
54,saurabh-dubey,Bowler,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Saurabh Dubey,20 lakh
66,aneeshwar-gautam,Bowling Allrounder,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Aneeshwar Gautam,20 lakh
208,rohan-rana,Allrounder,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Rohan Rana,20 lakh
229,sushant-mishra,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Sushant Mishra,20 lakh
263,vasu-vats,-,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Vasu Vats,20 lakh


In [87]:
df[df['Names'] == 'Matthew Wade']

,Player,Role,Mat,Inns,NO,Runs,HS,Ave,BF,SR,...,BBI,BBM,Ave,Econ,SR,4w,5w,10w,Names,BasePrice
46,matthew-wade,Wicketkeeper Batter,231,203,33,4559,130*,26.81,3360,135.68,...,-,-,-,-,-,-,-,-,Matthew Wade,2 crore


In [89]:
dff.to_csv('NameRetained.csv')

Thus, we scraped data from two websites one containing unsold and sold players, other containing retained and sold players and gathered all of their current T20 stats.